# Regression Prix Immobilier

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd


pd.set_option('display.max_columns', 5000)
pd.set_option("display.max_rows", 101)
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

# 📊 Analyse Simplifiée des Quartiers "Bad" (MAE Élevée)

Cette section contient l'analyse refactorisée et simplifiée pour identifier pourquoi certains quartiers ont une erreur de prédiction élevée (MAE). 

**Structure en 5 cellules comme demandé :**
1. **Contexte et préparation** : Chargement des données et définition des groupes de comparaison
2. **Métriques simples (ΔMAE) et tableaux récap** : Calcul des MAE pour les caractéristiques numériques et catégorielles  
3. **Graphiques lisibles pour les TOP_K features (numériques)** : Visualisation des médianes de SalePrice par quantile
4. **Graphiques lisibles pour les TOP_K features (catégorielles)** : Visualisation des moyennes de SalePrice par modalité
5. **Pistes d'action, par quartier "bad"** : Rapport textuel sur les caractéristiques qui pénalisent la MAE


In [ ]:
# ===== CELLULE 1 — CONTEXTE ET PRÉPARATION =====
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques pour plus de lisibilité
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

# Charger les données d'analyse
df = pd.read_csv("./data/kaggle_train_set.csv")
print(f"📊 Dataset chargé : {df.shape[0]} échantillons, {df.shape[1]} variables")

# Entraîner un modèle simple pour calculer les MAE par quartier
features_for_model = ['OverallQual', 'YearBuilt', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 
                     'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'YearRemodAdd', 'GarageArea', '1stFlrSF']

# Encoder les variables catégorielles pour le modèle
df_model = df.copy()
le_neighb = LabelEncoder()
le_exter = LabelEncoder() 
le_kitchen = LabelEncoder()

df_model['Neighborhood_enc'] = le_neighb.fit_transform(df_model['Neighborhood'])
df_model['ExterQual_enc'] = le_exter.fit_transform(df_model['ExterQual'])
df_model['KitchenQual_enc'] = le_kitchen.fit_transform(df_model['KitchenQual'])

X = df_model[features_for_model + ['Neighborhood_enc', 'ExterQual_enc', 'KitchenQual_enc']]
y = df_model['SalePrice']

# Diviser les données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entraîner le modèle
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Faire les prédictions
y_pred = model.predict(X_test)

# Calculer la MAE globale
mae_global = mean_absolute_error(y_test, y_pred)
print(f"🎯 MAE globale du modèle : {mae_global:,.0f}$")

# Créer un DataFrame avec les résultats de test
test_results = pd.DataFrame({
    'Neighborhood': df.iloc[X_test.index]['Neighborhood'].values,
    'SalePrice_true': y_test.values,
    'SalePrice_pred': y_pred,
    'residual': y_test.values - y_pred
})

# Calculer la MAE par quartier
mae_by_neighborhood = test_results.groupby('Neighborhood').apply(
    lambda x: mean_absolute_error(x['SalePrice_true'], x['SalePrice_pred'])
).sort_values(ascending=False)

print(f"\n🏘️  Quartiers avec MAE la plus élevée :")
print(mae_by_neighborhood.head(8))

# Définir les quartiers "bad" (MAE > percentile 75)
mae_threshold = mae_by_neighborhood.quantile(0.75)
bad_mae_region = mae_by_neighborhood[mae_by_neighborhood > mae_threshold].index.tolist()
print(f"\n🚨 Quartiers 'BAD' identifiés (MAE > {mae_threshold:,.0f}$) :")
for region in bad_mae_region:
    count = df[df['Neighborhood'] == region].shape[0]
    print(f"   - {region}: MAE = {mae_by_neighborhood[region]:,.0f}$ ({count} échantillons)")

# Créer le flag pour l'analyse comparative
bad_list = [n for n in bad_mae_region if n in df["Neighborhood"].unique()]
df["__is_bad__"] = df["Neighborhood"].isin(bad_list)

print(f"\n✅ Préparation terminée. {len(bad_list)} quartiers 'bad' identifiés pour analyse.")

In [ ]:
# ===== CELLULE 2 — MÉTRIQUES SIMPLES (ΔMAE) ET TABLEAUX RÉCAP =====

def mae(y_true, y_pred):
    """Fonction MAE simple"""
    return np.abs(y_true - y_pred).mean()

print("📋 CALCUL DES ΔMAE POUR TOUTES LES CARACTÉRISTIQUES\n")

# === FEATURES NUMÉRIQUES ===
num_features = ['OverallQual', 'YearBuilt', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 
                'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'YearRemodAdd', 'GarageArea', 
                'LotArea', '1stFlrSF']

# Calculer la prédiction "naïve" : médiane globale par quartile de feature
rows_num = []
for f in num_features:
    # Diviser en quartiles
    df['quartile'] = pd.qcut(df[f], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    
    # Pour chaque quartile, prédire avec la médiane du quartile
    predictions_out = []
    predictions_bad = []
    actual_out = []
    actual_bad = []
    
    for quartile in ['Q1', 'Q2', 'Q3', 'Q4']:
        quartile_data = df[df['quartile'] == quartile]
        median_price = quartile_data['SalePrice'].median()
        
        # Séparer quartiers "bad" vs "good"
        out_data = quartile_data[~quartile_data['__is_bad__']]
        bad_data = quartile_data[quartile_data['__is_bad__']]
        
        if len(out_data) > 0:
            predictions_out.extend([median_price] * len(out_data))
            actual_out.extend(out_data['SalePrice'].tolist())
        
        if len(bad_data) > 0:
            predictions_bad.extend([median_price] * len(bad_data))
            actual_bad.extend(bad_data['SalePrice'].tolist())
    
    # Calculer les MAE
    mae_out = mae(actual_out, predictions_out) if actual_out else 0
    mae_bad = mae(actual_bad, predictions_bad) if actual_bad else 0
    delta_mae = mae_bad - mae_out
    
    rows_num.append({
        "feature": f,
        "mae_out": mae_out,
        "mae_bad": mae_bad,
        "delta_mae": delta_mae
    })

num_summary = pd.DataFrame(rows_num).sort_values("delta_mae", ascending=False)

print("🔢 RÉSUMÉ FEATURES NUMÉRIQUES (Top 8 ΔMAE)")
print("="*60)
display(num_summary.head(8).round(0))

# === FEATURES CATÉGORIELLES ===
cat_features = ['ExterQual', 'KitchenQual', 'Neighborhood']

rows_cat = []
for f in cat_features:
    # Pour chaque modalité, prédire avec la moyenne de cette modalité
    predictions_out = []
    predictions_bad = []
    actual_out = []
    actual_bad = []
    
    for category in df[f].unique():
        cat_data = df[df[f] == category]
        mean_price = cat_data['SalePrice'].mean()
        
        # Séparer quartiers "bad" vs "good"
        out_data = cat_data[~cat_data['__is_bad__']]
        bad_data = cat_data[cat_data['__is_bad__']]
        
        if len(out_data) > 0:
            predictions_out.extend([mean_price] * len(out_data))
            actual_out.extend(out_data['SalePrice'].tolist())
        
        if len(bad_data) > 0:
            predictions_bad.extend([mean_price] * len(bad_data))
            actual_bad.extend(bad_data['SalePrice'].tolist())
    
    # Calculer les MAE
    mae_out = mae(actual_out, predictions_out) if actual_out else 0
    mae_bad = mae(actual_bad, predictions_bad) if actual_bad else 0
    delta_mae = mae_bad - mae_out
    
    rows_cat.append({
        "feature": f,
        "mae_out": mae_out,
        "mae_bad": mae_bad,
        "delta_mae": delta_mae
    })

cat_summary = pd.DataFrame(rows_cat).sort_values("delta_mae", ascending=False)

print("\n🏷️  RÉSUMÉ FEATURES CATÉGORIELLES")
print("="*60)
display(cat_summary.round(0))

print(f"\n💡 INSIGHTS ΔMAE :")
print(f"   • Feature numérique la plus problématique : {num_summary.iloc[0]['feature']} (ΔMAE: {num_summary.iloc[0]['delta_mae']:+,.0f}$)")
print(f"   • Feature catégorielle la plus problématique : {cat_summary.iloc[0]['feature']} (ΔMAE: {cat_summary.iloc[0]['delta_mae']:+,.0f}$)")

In [ ]:
# ===== CELLULE 3 — GRAPHIQUES LISIBLES POUR LES TOP_K FEATURES (NUMÉRIQUES) =====

TOP_K = 6  # Nombre de features à visualiser
top_num_features = num_summary.head(TOP_K)['feature'].tolist()

print(f"📊 VISUALISATION DES {TOP_K} FEATURES NUMÉRIQUES LES PLUS PROBLÉMATIQUES\n")

# Créer une grille de sous-graphiques compacte
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, feature in enumerate(top_num_features):
    ax = axes[idx]
    
    # Diviser en quartiles et calculer médianes
    df['quartile'] = pd.qcut(df[feature], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    
    # Données pour quartiers "bad" vs "good"
    quartile_medians = []
    quartiles = ['Q1', 'Q2', 'Q3', 'Q4']
    
    for q in quartiles:
        q_data = df[df['quartile'] == q]
        
        med_bad = q_data[q_data['__is_bad__']]['SalePrice'].median()
        med_good = q_data[~q_data['__is_bad__']]['SalePrice'].median()
        
        quartile_medians.append({
            'quartile': q,
            'median_bad': med_bad if not pd.isna(med_bad) else 0,
            'median_good': med_good if not pd.isna(med_good) else 0
        })
    
    plot_data = pd.DataFrame(quartile_medians)
    
    # Graphique en barres côte à côte
    x = np.arange(len(quartiles))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, plot_data['median_good']/1000, width, 
                   label='Quartiers "Good"', color='lightgreen', alpha=0.8)
    bars2 = ax.bar(x + width/2, plot_data['median_bad']/1000, width, 
                   label='Quartiers "Bad"', color='lightcoral', alpha=0.8)
    
    ax.set_xlabel(f'{feature} (par quartiles)')
    ax.set_ylabel('Prix médian (k$)')
    ax.set_title(f'{feature}\n(ΔMAE: {num_summary[num_summary["feature"] == feature]["delta_mae"].iloc[0]:+.0f}$)', 
                 fontsize=11, pad=10)
    ax.set_xticks(x)
    ax.set_xticklabels(quartiles)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    
    # Annoter les valeurs sur les barres
    for bar in bars1:
        height = bar.get_height()
        if height > 0:
            ax.text(bar.get_x() + bar.get_width()/2., height + 5,
                   f'{height:.0f}k', ha='center', va='bottom', fontsize=8)
    
    for bar in bars2:
        height = bar.get_height()
        if height > 0:
            ax.text(bar.get_x() + bar.get_width()/2., height + 5,
                   f'{height:.0f}k', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.suptitle(f'📈 Médianes SalePrice par Quartiles - TOP {TOP_K} Features Numériques Problématiques', 
             fontsize=14, y=1.02)
plt.show()

print("\n💡 INTERPRÉTATION :")
print("   • Les barres rouges (quartiers 'Bad') montrent des patterns différents")
print("   • Plus l'écart entre rouge et vert est important, plus la feature est problématique")
print("   • Cela explique pourquoi la MAE est élevée dans ces quartiers")

In [ ]:
# ===== CELLULE 4 — GRAPHIQUES LISIBLES POUR LES TOP_K FEATURES (CATÉGORIELLES) =====

print(f"📊 VISUALISATION DES FEATURES CATÉGORIELLES PROBLÉMATIQUES\n")

# Prendre toutes les features catégorielles
top_cat_features = cat_summary['feature'].tolist()

fig, axes = plt.subplots(1, len(top_cat_features), figsize=(15, 6))
if len(top_cat_features) == 1:
    axes = [axes]

for idx, feature in enumerate(top_cat_features):
    ax = axes[idx]
    
    # Calculer moyennes par modalité
    modalite_means = []
    categories = df[feature].unique()
    
    for cat in categories:
        cat_data = df[df[feature] == cat]
        
        mean_bad = cat_data[cat_data['__is_bad__']]['SalePrice'].mean()
        mean_good = cat_data[~cat_data['__is_bad__']]['SalePrice'].mean()
        count_bad = cat_data[cat_data['__is_bad__']].shape[0]
        count_good = cat_data[~cat_data['__is_bad__']].shape[0]
        
        modalite_means.append({
            'category': cat,
            'mean_bad': mean_bad if not pd.isna(mean_bad) else 0,
            'mean_good': mean_good if not pd.isna(mean_good) else 0,
            'count_bad': count_bad,
            'count_good': count_good
        })
    
    plot_data = pd.DataFrame(modalite_means)
    # Trier par différence pour une meilleure lisibilité
    plot_data['diff'] = plot_data['mean_bad'] - plot_data['mean_good']
    plot_data = plot_data.sort_values('diff', ascending=True)
    
    # Ne garder que les modalités avec suffisamment de données
    plot_data = plot_data[(plot_data['count_bad'] >= 3) | (plot_data['count_good'] >= 3)]
    
    if len(plot_data) > 0:
        x = np.arange(len(plot_data))
        width = 0.35
        
        bars1 = ax.bar(x - width/2, plot_data['mean_good']/1000, width,
                       label='Quartiers "Good"', color='lightgreen', alpha=0.8)
        bars2 = ax.bar(x + width/2, plot_data['mean_bad']/1000, width,
                       label='Quartiers "Bad"', color='lightcoral', alpha=0.8)
        
        ax.set_xlabel(f'Modalités de {feature}')
        ax.set_ylabel('Prix moyen (k$)')
        ax.set_title(f'{feature}\n(ΔMAE: {cat_summary[cat_summary["feature"] == feature]["delta_mae"].iloc[0]:+.0f}$)', 
                     fontsize=11, pad=10)
        ax.set_xticks(x)
        ax.set_xticklabels(plot_data['category'], rotation=45 if len(plot_data) > 4 else 0)
        ax.legend(fontsize=9)
        ax.grid(True, alpha=0.3)
        
        # Annoter les différences importantes
        for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
            height1 = bar1.get_height()
            height2 = bar2.get_height()
            diff = abs(height2 - height1)
            
            if diff > 20:  # Différence > 20k$
                max_height = max(height1, height2)
                ax.text(x[i], max_height + 10, f'Δ{diff:.0f}k',
                       ha='center', va='bottom', fontsize=8, color='red', weight='bold')

plt.tight_layout()
plt.suptitle('📊 Prix Moyens par Modalité - Features Catégorielles Problématiques', 
             fontsize=14, y=1.02)
plt.show()

print("\n💡 INTERPRÉTATION :")
print("   • Les annotations rouges 'Δ' montrent les écarts importants (>20k$)")
print("   • Ces écarts expliquent pourquoi le modèle peine sur les quartiers 'Bad'")
print("   • Les mêmes modalités ont des prix très différents selon le quartier")

In [ ]:
# ===== CELLULE 5 — PISTES D'ACTION, PAR QUARTIER "BAD" =====

print("🚀 RAPPORT D'ACTIONS PAR QUARTIER 'BAD'\n")
print("="*80)

for quartier in bad_list:
    print(f"\n🏘️  QUARTIER : {quartier}")
    print("="*50)
    
    # Données du quartier
    quartier_data = df[df['Neighborhood'] == quartier]
    mae_quartier = mae_by_neighborhood[quartier]
    n_samples = len(quartier_data)
    
    print(f"📊 MAE: {mae_quartier:,.0f}$ | Échantillons: {n_samples} | Écart vs MAE globale: {mae_quartier - mae_global:+,.0f}$")
    
    # Analyser les top features problématiques pour ce quartier
    print("\n🔍 CARACTÉRISTIQUES PROBLÉMATIQUES :")
    
    # Top 3 features numériques
    print("\n   📈 Features numériques :")
    for feature in top_num_features[:3]:
        quartier_median = quartier_data[feature].median()
        global_median = df[feature].median()
        diff_pct = ((quartier_median - global_median) / global_median * 100) if global_median != 0 else 0
        
        # Quartile du quartier par rapport à la population globale
        quartile_pos = pd.qcut(df[feature], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
        quartier_quartile = pd.qcut(df[feature], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4']).iloc[quartier_data.index[0]]
        
        delta_mae_feature = num_summary[num_summary['feature'] == feature]['delta_mae'].iloc[0]
        
        print(f"      • {feature}: médiane {quartier_median:.0f} ({diff_pct:+.1f}% vs global) ")
        print(f"        → Position: {quartier_quartile} | Impact ΔMAE: {delta_mae_feature:+,.0f}$")
    
    # Features catégorielles
    print("\n   🏷️  Features catégorielles :")
    for feature in top_cat_features:
        if feature != 'Neighborhood':  # Skip neighborhood car c'est le quartier analysé
            # Distribution des modalités dans ce quartier
            quartier_dist = quartier_data[feature].value_counts(normalize=True)
            global_dist = df[feature].value_counts(normalize=True)
            
            # Modalité la plus fréquente dans ce quartier
            top_modalite = quartier_dist.index[0]
            quartier_freq = quartier_dist.iloc[0] * 100
            global_freq = global_dist.get(top_modalite, 0) * 100
            
            delta_mae_feature = cat_summary[cat_summary['feature'] == feature]['delta_mae'].iloc[0]
            
            print(f"      • {feature}: modalité dominante '{top_modalite}' ({quartier_freq:.0f}% vs {global_freq:.0f}% global)")
            print(f"        → Impact ΔMAE: {delta_mae_feature:+,.0f}$")
    
    # Recommandations spécifiques
    print("\n💡 RECOMMANDATIONS :")
    
    # Recommandation basée sur la feature la plus problématique
    top_problematic = num_summary.iloc[0]['feature']
    top_delta = num_summary.iloc[0]['delta_mae']
    
    if top_delta > 5000:
        print(f"   1. 🎯 PRIORITÉ HAUTE: Améliorer la modélisation pour '{top_problematic}'")
        print(f"      → Ajouter des interactions spécifiques au quartier {quartier}")
        
    if mae_quartier > mae_global * 1.5:
        print(f"   2. 📊 Collecter plus de données pour ce quartier (seulement {n_samples} échantillons)")
        
    if len(quartier_data) < 20:
        print(f"   3. ⚠️  Quartier sous-représenté: considérer un regroupement avec quartiers similaires")
    
    # Recommandation basée sur les features catégorielles
    top_cat_problematic = cat_summary.iloc[0]['feature'] 
    if cat_summary.iloc[0]['delta_mae'] > 3000:
        print(f"   4. 🏷️  Feature catégorielle '{top_cat_problematic}': créer encodage spécifique au quartier")
    
    print(f"   5. 🔧 Envisager un modèle spécialisé pour les quartiers à haute variabilité")

print(f"\n\n📋 RÉSUMÉ GLOBAL DES ACTIONS")
print("="*80)
print(f"🎯 {len(bad_list)} quartiers identifiés comme problématiques")
print(f"📊 Feature numérique prioritaire: {num_summary.iloc[0]['feature']} (ΔMAE: {num_summary.iloc[0]['delta_mae']:+,.0f}$)")
print(f"🏷️  Feature catégorielle prioritaire: {cat_summary.iloc[0]['feature']} (ΔMAE: {cat_summary.iloc[0]['delta_mae']:+,.0f}$)")
print(f"\n✅ Analyse terminée. Prochaines étapes: implémenter les améliorations suggérées.")